In [1]:
from sympy import symbols, diff, solve, Matrix, Eq
from sympy import sin, cos, pi

x,y,z = symbols('x y z', real=True)
a = symbols('a', real=True)
r = symbols('r', positive=True)
l = symbols('lambda', real=True)

f = 6 + x**2 - y**2 + 2*x*y

## (a) Altura de l'edifici

El problema és calcular el màxim de $f(x,y) = 6 + x^2 - y^2 + 2xy$ dins del compacte $R = \{(x,y)\mid x^2 + y^2 \leq4\}$. 

Anem pas a pas:

1. Estudiem els posibles extrems a l'interior de $R$

2. Estudiem els possibles extrems a la frontera de $R$.
   Aquest segon cas es pot fer de dues maneres que detallarem d'aquí a un mooment.
   
   
### Estudi de l'interior

Busquem els _punts crítics_ imposant $\nabla f = \vec 0$:

In [2]:
[diff(f,v) for v in (x,y)]

[2*x + 2*y, 2*x - 2*y]

això correspon a un sistema lineal amb solució única:

In [3]:
sols = []
[s] = solve(_,dict=True); print s
sols.append(s)

{y: 0, x: 0}


### Estudi de la frontera

El problema a la frontera està condicionat per $x^2 + y^2 = 4$ i, com deiem, es pot fer de dues maneres:

#### Pel mètode dels _multiplicadors de Lagrange_

Posant com a _Lagrangiana_: $$L = \lambda \left(x^{2} + y^{2} - 4\right) + x^{2} + 2 x y - y^{2} + 6$$

In [4]:
L = f + l*(x**2 + y**2 - 4)
eqs = [diff(L,c).factor() for c in (x,y,l)]
eqs

[2*(lambda*x + x + y), 2*(lambda*y + x - y), x**2 + y**2 - 4]

Observeu que la primera i segona equació formen un sistema linieal en $x$ i $y$ (deixant $\lambda$ com a paràmetre a determinar).

Aquest sistema posat en forma matricial és:

In [5]:
Matrix([[diff(c,v) for v in (x,y)] for c in eqs[:2]])

Matrix([
[2*lambda + 2,            2],
[           2, 2*lambda - 2]])

i el sistema és singular si i només si el determinant s'anul·la:

In [6]:
_.det()

4*lambda**2 - 8

In [7]:
solve(_, dict=True)

[{lambda: -sqrt(2)}, {lambda: sqrt(2)}]

Estudiem per separat les dues possibilitats:

* El sub-sistema és singular ($\lambda=\pm\sqrt2$):


   En aquest cas el sistema lineal de les 2 primeres equacions és indeterminat i podem eliminar una de les dues equacions (ja que serà linealment dependent de l'altra). Aquí resolem la primera equació i el resultat el substituim a la tercera:

In [8]:
for u in _:
    [s] = solve(eqs[0].subs(u), dict=True)
    u.update(s)
    e = eqs[-1].subs(u)
    for t in solve(e,y,dict=True):
        u.update(t)
        assert all(e.subs(u).simplify()==0 for e in eqs)
        print u
        sols.append(u)

{y: -sqrt(2)/sqrt(sqrt(2) + 2), x: y + sqrt(2)*y, lambda: -sqrt(2)}
{y: sqrt(2)/sqrt(sqrt(2) + 2), x: y + sqrt(2)*y, lambda: -sqrt(2)}
{y: -sqrt(2)/sqrt(-sqrt(2) + 2), x: -sqrt(2)*y + y, lambda: sqrt(2)}
{y: sqrt(2)/sqrt(-sqrt(2) + 2), x: -sqrt(2)*y + y, lambda: sqrt(2)}


* Cas no-singular ($\lambda^2 \neq2$):

   En aquest cas el sistema lineal de dalt només té la solució $x=y=0$, però això no compleix la darrera equació:

In [9]:
sol = {x:0, y:0}
[e.subs(sol) for e in eqs]

[0, 0, -4]

 Tenim el _punt crític_ de l'interior i 4 punts més que hem trobat a la frontera.
 Ara avaluem $z$ a cada un d'aquests punts i ens quedem amb el valor més gran:

In [10]:
for s in sols:
    p = Matrix([x,y]).subs(s)
    p.simplify()
    zval = f.subs(s).simplify()
    print tuple(p), zval, float(zval)

(0, 0) 6 6.0
(sqrt(sqrt(2) + 2), sqrt(2)/sqrt(sqrt(2) + 2)) 4*sqrt(2) + 6 11.6568542495
(sqrt(sqrt(2) + 2), sqrt(2)/sqrt(sqrt(2) + 2)) 4*sqrt(2) + 6 11.6568542495
(sqrt(-sqrt(2) + 2), sqrt(2)/sqrt(-sqrt(2) + 2)) -4*sqrt(2) + 6 0.343145750508
(sqrt(-sqrt(2) + 2), sqrt(2)/sqrt(-sqrt(2) + 2)) -4*sqrt(2) + 6 0.343145750508


L'altura màxima serà:

In [11]:
zmax = f.subs(sols[1]).simplify()
print zmax
print float(zmax)

4*sqrt(2) + 6
11.6568542495


#### Solució alternativa: Parametritzant la frontera

Com que la condició $x^2+y^2=4$ correspon a una circumferéncia de radi 2, podem parametrizar-ho com una corba:

In [12]:
rt = {x: 2*cos(a), y: 2*sin(a)}
h = f.subs(rt).simplify()
h

4*sqrt(2)*sin(2*a + pi/4) + 6

Ara l'altura $h$ és funció de l'angle $a$. Derivem, igualem a zero i resolem l'equació resultant:

In [13]:
Eq(h.diff(a), 0)

8*sqrt(2)*cos(2*a + pi/4) == 0

In [14]:
solve(_,a)

[pi/8, 5*pi/8]

Aquí el _solver_ s'ha quedat una mica curt perquè no ha tingut em compte que el cosinus és periòdic.
Afegim el que falta:

In [15]:
_ + [c + pi for c in _ if c+pi < 2*pi]

[pi/8, 5*pi/8, 9*pi/8, 13*pi/8]

Calculem $h$ en aquests valors de $a$ i ens quedem amb el més gran:

In [16]:
for u in _:
    print tuple(Matrix([x,y]).subs(rt).subs(a,u)), float(h.subs(a,u))

(2*sqrt(sqrt(2)/4 + 1/2), 2*sqrt(-sqrt(2)/4 + 1/2)) 11.6568542495
(-2*sqrt(-sqrt(2)/4 + 1/2), 2*sqrt(sqrt(2)/4 + 1/2)) 0.343145750508
(-2*sqrt(sqrt(2)/4 + 1/2), -2*sqrt(-sqrt(2)/4 + 1/2)) 11.6568542495
(2*sqrt(-sqrt(2)/4 + 1/2), -2*sqrt(sqrt(2)/4 + 1/2)) 0.343145750508


## (b) Àrea de la coberta

Per a calcular l'àrea primer hem de parametritzar la coberta. Usem coordenades polars ja que la planta és un cercle:

In [17]:
from sympy import integrate
R ={x: r*cos(a), y: r*sin(a), z: f}
R.update({z: R[z].subs(R).simplify()})
R

{y: r*sin(a), x: r*cos(a), z: sqrt(2)*r**2*sin(2*a + pi/4) + 6}

Calculem el _producte vectorial fonamental_ (aquí `dR`) i després la seva norma:

In [18]:
from sympy import powdenest

Rr = Matrix([diff(c.subs(R), r) for c in (x,y,z)]); print Rr
Ra = Matrix([diff(c.subs(R), a) for c in (x,y,z)]); print Ra
dR = powdenest(Rr.cross(Ra)).simplify(); print dR
dR.norm().simplify()

Matrix([[cos(a)], [sin(a)], [2*sqrt(2)*r*sin(2*a + pi/4)]])
Matrix([[-r*sin(a)], [r*cos(a)], [2*sqrt(2)*r**2*cos(2*a + pi/4)]])
Matrix([
[-2*sqrt(2)*r**2*sin(a + pi/4)],
[-2*sqrt(2)*r**2*cos(a + pi/4)],
[                            r]])


r*sqrt(8*r**2 + 1)

... i integrem primer per a $0\leq r\leq2$ i després per a $a\in[0,2\pi]$:

In [19]:
integrate(integrate(_, (r,0,2)), (a,0,2*pi)).simplify()

pi*(-1 + 33*sqrt(33))/12

In [20]:
float(_)

49.36765908543893

#### Solució alternativa: Usnat coordenades cartesianes

Recordem de teoria que la fòrmula $(-f_x,-f_y,1)$ ens donava el producte vectorial fonamental d'una superfície que era el graf d'una funció $f(x,y)$:

In [21]:
Matrix([-diff(f,x), -diff(f,y), 1])

Matrix([
[-2*x - 2*y],
[-2*x + 2*y],
[         1]])

La seva norma serà:

In [22]:
_.norm().simplify()

sqrt(8*x**2 + 8*y**2 + 1)

Surt a compte fer un canvi de variable a polars:

In [23]:
_.subs({x: r*cos(a), y: r*sin(a)}).simplify()

sqrt(8*r**2 + 1)

In [24]:
integrate(integrate(_*r, (r,0,2)), (a,0,2*pi)).simplify()

pi*(-1 + 33*sqrt(33))/12

## (c) Àrea lateral

Paramatritzem ara la superfície lateral de l'edifici. Està sobre la circumferència de radi 2 i $0\leq z\leq f(x,y)$.
Usarem l'angle $a$ i $z$ com a paràmetres i calcularem el producte vectorial fonamental:

In [25]:
S = {x: 2*cos(a), y:2*sin(a)}
Sa = Matrix([diff(c.subs(S), a) for c in (x,y,z)]); print Sa
Sz = Matrix([diff(c.subs(S), z) for c in (x,y,z)]); print Sz
dS = Sa.cross(Sz)

Matrix([[-2*sin(a)], [2*cos(a)], [0]])
Matrix([[0], [0], [1]])


... que té norma:

In [26]:
dSn = dS.norm().simplify(); dSn

2

Ara integrem. Com deiem $z$ varia des de 0 (planta) fins a $f(x,y)$:

In [27]:
zmax = f.subs(S); zmax.simplify()

4*sqrt(2)*sin(2*a + pi/4) + 6

In [28]:
integrate(dSn, (z,0,zmax))

-8*sin(a)**2 + 16*sin(a)*cos(a) + 8*cos(a)**2 + 12

Finalment integrem respecte de $a$:

In [29]:
areaL = integrate(_, (a,0,2*pi)); areaL

24*pi

## (d) Flux de la coberta

Ara ens demanen calcula el flux de $\vec F(x,y,z) = (x,y,z)$ sobre la coberta.
Això vol dir calcular $$\int_C \vec F\cdot d\vec S$$.

Hi ha dues maneres de fer-ho: 

* Directament.

* Usant el **teorema de la divergència**.

### I. Directament

Recordem que, per definició, el flux és igual a
$$\iint_{[0,2]\times[0,2\pi]} \vec F(\vec R(r,a))\cdot(\vec R_r\times \vec R_a)\,dr\,da.$$

In [30]:
F = Matrix([x,y,z])
F.subs(R)

Matrix([
[                        r*cos(a)],
[                        r*sin(a)],
[sqrt(2)*r**2*sin(2*a + pi/4) + 6]])

El producte vectorial ja l'haviém calculat abans:

In [31]:
dR.simplify()

Matrix([
[-2*sqrt(2)*r**2*sin(a + pi/4)],
[-2*sqrt(2)*r**2*cos(a + pi/4)],
[                            r]])

La funció a integrar és:

In [32]:
F.subs(R).dot(dR).simplify()

r*(-sqrt(2)*r**2*sin(2*a + pi/4) + 6)

In [33]:
integrate(_, (r,0,2))

-4*sqrt(2)*sin(2*a + pi/4) + 12

In [34]:
integrate(_, (a,0,2*pi))

24*pi

#### Alternativament: Usant coordenades cartesianes

In [35]:
XY = F.subs(z,f); XY

Matrix([
[                      x],
[                      y],
[x**2 + 2*x*y - y**2 + 6]])

In [36]:
XYx = Matrix([diff(c, x) for c in XY]); print XYx
XYy = Matrix([diff(c, y) for c in XY]); print XYy
XYx.cross(XYy)

Matrix([[1], [0], [2*x + 2*y]])
Matrix([[0], [1], [2*x - 2*y]])


Matrix([
[-2*x - 2*y],
[-2*x + 2*y],
[         1]])

In [37]:
F.subs(z,f).dot(_).simplify()

-x**2 - 2*x*y + y**2 + 6

Com que ho hem d'integrar sobre una regió circular, canviem a polars (no oblideu el Jacobià `r`!)

In [38]:
(_.subs({x: r*cos(a), y: r*sin(a)})*r)

r*(r**2*sin(a)**2 - 2*r**2*sin(a)*cos(a) - r**2*cos(a)**2 + 6)

In [39]:
integrate(_, (r,0,2))

4*sin(a)**2 - 8*sin(a)*cos(a) - 4*cos(a)**2 + 12

In [40]:
integrate(_, (a,0,2*pi))

24*pi

### II. Usant el teorema de la divergència

Aquest teorema ens diu que:
$$\iiint_E \nabla\cdot\vec F = \int_C\vec F\cdot d\vec S + \int_L\vec F\cdot d\vec S + \int_P\vec F\cdot d\vec S;$$
on $E$ designa l'edifici sencer i $C$, $L$ i $P$ són les tres parts de la frontera de $E$ orientades amb el vector normal exterior (Això és: coberta, lateral i planta).

#### II.a) Part de la divergència

La _divergència_ de $\vec F$ és:

In [41]:
divF = sum(diff(c,u) for c,u in zip(F,[x,y,z])); divF

3

La integral triple la fem començant per $z$:

In [42]:
integrate(divF, (z,0,f))

3*x**2 + 6*x*y - 3*y**2 + 18

... passem a polars:

In [43]:
_.subs({x: r*cos(a), y:r*sin(a)}).simplify()

3*sqrt(2)*r**2*sin(2*a + pi/4) + 18

... i integrem respecte de $r$ (amb el jacobià, és clar!):

In [44]:
integrate(_*r, (r,0,2))

12*sqrt(2)*sin(2*a + pi/4) + 36

Finalment integrem respecte de l'angle:

In [45]:
Fdiv = integrate(_, (a,0,2*pi)); Fdiv

72*pi

#### II. b) Part de l'area lateral

La parametrització i el producte fonamental ja el teniem d'abans. Només canvia la funció que estem integrant:

In [46]:
print S
print dS
F.subs(S).dot(dS).simplify()

{y: 2*sin(a), x: 2*cos(a)}
Matrix([[2*cos(a)], [2*sin(a)], [0]])


4

In [47]:
integrate(_, (z,0,f.subs(S)))

-16*sin(a)**2 + 32*sin(a)*cos(a) + 16*cos(a)**2 + 24

In [48]:
FL = integrate(_, (a,0,2*pi)); FL

48*pi

#### II.c) Planta

Parametritzem el cercle de la planta en polars i calculem el producte fonamental.

In [49]:
T = {x: r*cos(a), y: r*sin(a), z: 0}
Tr = Matrix([diff(c.subs(T),r) for c in (x,y,z)]); print Tr
Ta = Matrix([diff(c.subs(T),a) for c in (x,y,z)]); print Ta
dT = Tr.cross(Ta)
dT.simplify(); print dT

Matrix([[cos(a)], [sin(a)], [0]])
Matrix([[-r*sin(a)], [r*cos(a)], [0]])
Matrix([[0], [0], [r]])


Observeu que el producte fonamental apunta cap a l'interior de $E$ (ja que la tercera component $r\geq0$).
Això vol dir que veritablement estem parametritzant $-P$ i ho haurem de tenir en compte quan fem el balanç de tots els fluxos.

Ara vegem què és el que hem d'integrar:

In [50]:
F.subs(T).dot(dT)

0

... i no cal integrar res més perquà val 0.

Ja tenim tots els fluxos menys el que volem (el de $C$) pasant-los a l'altra banda de la igualtat, podem dir que el flux que ens demanen és:

In [51]:
Fdiv -FL

24*pi